In [1]:
#!/usr/bin/env bash

<span style="color: red; font-weight: bold">Use the following command to launch a server for this notebook:</span>

```bash
git clone git@github.com:input-output-hk/marlowe-cardano.git
cd marlowe-cardano/marlowe-runtime/
nix run ../marlowe-cli
```

Then navigate to the `examples/` folder in Jupyter and open this notebook.

# Demonstrating the Marlowe Transaction Deposit Component of Marlowe Runtime Using Role Tokens

## Preliminaries

Record version numbers

In [69]:
marlowe-cli --version

marlowe-cli 0.0.10.0


In [70]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [71]:
git rev-parse HEAD

a57b40407e8efed9c926695448efd8b518d54f1c


### Setup the faucet.

Set the location of keys.

In [72]:
TREASURY=treasury

Set the faucet.

In [73]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDR"

addr_test1vq2a4cdym9mn38nf894p5sjqurns4ehjm3pt5t8l4djcnzqjzztcr


### Select network

In a separate terminal, set up a tunnel to the Marlowe Runtime development server:
```bash
rm /tmp/preview.socket
ssh -NT \
  -L/tmp/preview.socket:/data/networks/preview/node.socket \
  -L 3717:127.0.0.1:23717 \
  -L 3718:127.0.0.1:23718 \
  -L 3719:127.0.0.1:23719 \
  -L 3721:127.0.0.1:23721 \
  -L 3723:127.0.0.1:23723 \
  54.202.238.5                                              
```

In [74]:
export CARDANO_NODE_SOCKET_PATH=/tmp/preview.socket
export CARDANO_TESTNET_MAGIC=2
MAGIC=(--testnet-magic 2)
echo "${MAGIC[@]}"

--testnet-magic 2


### Check that the reference script has been published

Check that the Marlowe semantics validator was published.

In [75]:
marlowe-cli transaction find-published


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2"
    }
}


### Participants

#### The Party

In [76]:
PARTY_SKEY="$TREASURY/john-fletcher.skey"
PARTY_VKEY="$TREASURY/john-fletcher.vkey"

Create the first party's keys, if necessary.

In [77]:
if [[ ! -e "$PARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_SKEY" --verification-key-file "$PARTY_VKEY"
fi
PARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PARTY_VKEY")
echo "$PARTY_ADDR"

addr_test1vrwd29434q5sxe9wq5xplc0ehgqyehpjujx4kvdcwpck2kgp8qjs9


Fund the address.

In [78]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600 \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$PARTY_ADDR"

TxId "2d1a7b08ba0ef004e0a5d6114ecf34f9b816ffe55499c1805e9959d44b8c8f08"


#### The Counterparty

In [79]:
COUNTERPARTY_SKEY="$TREASURY/thomas-kyd.skey"
COUNTERPARTY_VKEY="$TREASURY/thomas-kyd.vkey"

Create the second party's keys, if necessary.

In [80]:
if [[ ! -e "$COUNTERPARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$COUNTERPARTY_SKEY" --verification-key-file "$COUNTERPARTY_VKEY"
fi
COUNTERPARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$COUNTERPARTY_VKEY")
echo "$COUNTERPARTY_ADDR"

addr_test1vq9wf6m58ga0yg62w5yl0rjzt8c4updtt7ttln570rwz09c64kcd5


Fund the address.

In [81]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600  \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$COUNTERPARTY_ADDR"

TxId "9b9cb00e41f602fbdad226010342614a233294d0ccf80387c6b584bf9f8fdc18"


### Time computations

In [82]:
SECOND=1000
MINUTE=$((60 * SECOND))

In [83]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1666983720000


## The Contract

We set the parameters for the ACTUS PAM contract.

In [84]:
MINIMUM_ADA=3000000

FIXED_POINT=1000000
PRINCIPAL=100
INTEREST_RATE=0.02
INTEREST=$(jq -n $PRINCIPAL*$INTEREST_RATE)

STATUS_DATE=$(date -d "$(date -u -R -d @$((NOW/1000)))" +"%Y-%m-%dT00:00:00")
INITIAL_EXCHANGE_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 1 year" +"%Y-01-01T00:00:00")
MATURITY_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 2 year" +"%Y-01-01T00:00:00")

In [85]:
yaml2json << EOI > deposit.actus
scheduleConfig:
  businessDayConvention: "NULL"
  endOfMonthConvention: "EOM"
  calendar: "NC"
maturityDate: "$MATURITY_DATE"
contractId: "0"
enableSettlement: false
initialExchangeDate: "$INITIAL_EXCHANGE_DATE"
contractRole: "RPA"
penaltyType: "O"
cycleAnchorDateOfInterestPayment: "$INITIAL_EXCHANGE_DATE"
contractType: "PAM"
notionalPrincipal: $PRINCIPAL
contractPerformance: "PF"
collateralAmount: 0
dayCountConvention: "30E360"
accruedInterest: 0
statusDate: "$STATUS_DATE"
cycleOfInterestPayment: "P1YL1"
prepaymentEffect: "N"
nominalInterestRate: $INTEREST_RATE
interestCalculationBase: "NT"
EOI
cat deposit.actus

{"accruedInterest":0,"collateralAmount":0,"contractId":"0","contractPerformance":"PF","contractRole":"RPA","contractType":"PAM","cycleAnchorDateOfInterestPayment":"2023-01-01T00:00:00","cycleOfInterestPayment":"P1YL1","dayCountConvention":"30E360","enableSettlement":false,"initialExchangeDate":"2023-01-01T00:00:00","interestCalculationBase":"NT","maturityDate":"2024-01-01T00:00:00","nominalInterestRate":0.02,"notionalPrincipal":100,"penaltyType":"O","prepaymentEffect":"N","scheduleConfig":{"businessDayConvention":"NULL","calendar":"NC","endOfMonthConvention":"EOM"},"statusDate":"2022-10-28T00:00:00"}


Create the contract.

In [86]:
marlowe-cli template actus \
  --minimum-ada "$MINIMUM_ADA" \
  --party Lender \
  --counter-party Borrower \
  --actus-terms-file  deposit.actus \
  --out-contract-file deposit-1.contract \
  --out-state-file /dev/null

Since we are testing, we don't really want to wait months or years for timeouts, so we edit the contract file to change the maturity date to 15 minutes from now and the initial deposit to 10 minutes from now.

In [87]:
sed -i \
  -e "s/$(jq '.timeout' deposit-1.contract)/$((NOW + 10 * MINUTE))/" \
  -e "s/$(($(date -d "$MATURITY_DATE" -u +%s) * SECOND))/$((NOW + 15 * MINUTE))/" \
  deposit-1.contract

View the contract.

In [88]:
json2yaml deposit-1.contract

timeout: 1666984320000
timeout_continuation: close
when:
- case:
    deposits:
      negate:
        negate: 100000000
    into_account:
      role_token: Lender
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: Lender
  then:
    from_account:
      role_token: Lender
    pay: 100000000
    then:
      timeout: 1666984620000
      timeout_continuation: close
      when:
      - case:
          deposits: 2000000
          into_account:
            role_token: Borrower
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            role_token: Borrower
        then:
          from_account:
            role_token: Borrower
          pay: 2000000
          then:
            timeout: 1666984620000
            timeout_continuation: close
            when:
            - case:
                deposits: 100000000
                into_account:
                  role_token: Borrower
                of_token:
    

## Run the Contract

### Transaction 1. Create the contract

Build the transaction.

In [89]:
TX_1=$(
marlowe create \
  --core-file deposit-1.contract \
  --role "Lender=$PARTY_ADDR" \
  --role "Borrower=$COUNTERPARTY_ADDR" \
  --min-utxo "$MINIMUM_ADA" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign deposit-1.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
CONTRACT_ID="$TX_1"
echo "CONTRACT_ID = TX_1 = $CONTRACT_ID"

CONTRACT_ID = TX_1 = 7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253#1


Sign the transaction.

In [90]:
cardano-cli transaction sign \
  --tx-body-file deposit-1.txbody \
  --out-file     deposit-1.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [91]:
marlowe submit deposit-1.tx

"{\"blockHeaderHash\":\"51785845562e3493647637e398775d62f70bff778b2793e0787acb6c551f8c2a\",\"blockNo\":322731,\"slotNo\":6980558}"


View the contract's UTxO.

In [92]:
CONTRACT_ADDR=$(marlowe-cli contract address)
echo "$CONTRACT_ADDR"

addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv


In [93]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_1//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "dca9a9e18a071e1339d97209a2c8e0e70d53a00a639cff5b3c46ec53270aae10"


View the party's UTxOs.

In [94]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253     0        245412905 lovelace + TxOutDatumNone
7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253     3        1034400 lovelace + 1 42e84366ff974676655faf4f99f917145a12d29335ecb8162e6f843c.4c656e646572 + TxOutDatumNone
7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253     4        1034400 lovelace + 1 567a873dff9bad13eb231829ebe26ad545862078c5251b8ef84dac9f.4c656e646572 + TxOutDatumNone


View the counterparty's UTxOs.

In [95]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253     2        1043020 lovelace + 1 42e84366ff974676655faf4f99f917145a12d29335ecb8162e6f843c.426f72726f776572 + TxOutDatumNone
9b9cb00e41f602fbdad226010342614a233294d0ccf80387c6b584bf9f8fdc18     1        250000000 lovelace + TxOutDatumNone
b2a9bca0c5481983ea8c63aa78a29b91ef29f608e50b48553b3f5c8e88a33d1c     1        2000000 lovelace + 1 567a873dff9bad13eb231829ebe26ad545862078c5251b8ef84dac9f.426f72726f776572 + TxOutDatumNone


Watch the contract.

In [96]:
marlowe add "$CONTRACT_ID"

7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253#1


View the contract's history.

In [97]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253 (creation)
ContractId:      7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253#1
SlotNo:          6980558
BlockNo:         322731
BlockId:         51785845562e3493647637e398775d62f70bff778b2793e0787acb6c551f8c2a
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Role "Lender" Role "Lender"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Role "Lender"
            (Party Role "Borrower")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
                  (Deposit Role "Borrower" Role "Borrower"
                     (Token "" "")
                     (Constant 2000000))
                  (Pay Role "Borrower"
                     (Party Role "Lender")
                     (Token "" 

### Transaction 2. Party deposits loan amount

The party deposits the loan amount.

In [98]:
TX_2=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party Lender \
  --to-party Lender \
  --lovelace "$((PRINCIPAL*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign deposit-2.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_2 = $TX_2"

TX_2 = e4dd26d440107de4af57274f640bde39bebd0364ceb43fbdaf86346187bf3736


Sign the transaction.

In [99]:
cardano-cli transaction sign \
  --tx-body-file deposit-2.txbody \
  --out-file     deposit-2.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [100]:
marlowe submit deposit-2.tx

"{\"blockHeaderHash\":\"4f0af30bb59acfdad8a1cb47c75457dfb6826f732db765f22365a1df42dabacd\",\"blockNo\":322733,\"slotNo\":6980594}"


View the contract's UTxO.

In [101]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_2//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
e4dd26d440107de4af57274f640bde39bebd0364ceb43fbdaf86346187bf3736     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "1bb9acb2765628e56332db24a3ecdb04f02a03213320dfb71708bf74864ae4e3"


View the party's UTxOs.

In [102]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253     4        1034400 lovelace + 1 567a873dff9bad13eb231829ebe26ad545862078c5251b8ef84dac9f.4c656e646572 + TxOutDatumNone
e4dd26d440107de4af57274f640bde39bebd0364ceb43fbdaf86346187bf3736     0        144586942 lovelace + TxOutDatumNone
e4dd26d440107de4af57274f640bde39bebd0364ceb43fbdaf86346187bf3736     2        1034400 lovelace + 1 42e84366ff974676655faf4f99f917145a12d29335ecb8162e6f843c.4c656e646572 + TxOutDatumNone


View the counterparty's UTxOs.

In [103]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253     2        1043020 lovelace + 1 42e84366ff974676655faf4f99f917145a12d29335ecb8162e6f843c.426f72726f776572 + TxOutDatumNone
9b9cb00e41f602fbdad226010342614a233294d0ccf80387c6b584bf9f8fdc18     1        250000000 lovelace + TxOutDatumNone
b2a9bca0c5481983ea8c63aa78a29b91ef29f608e50b48553b3f5c8e88a33d1c     1        2000000 lovelace + 1 567a873dff9bad13eb231829ebe26ad545862078c5251b8ef84dac9f.426f72726f776572 + TxOutDatumNone


View the contract's history.

In [104]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253 (creation)
ContractId:      7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253#1
SlotNo:          6980558
BlockNo:         322731
BlockId:         51785845562e3493647637e398775d62f70bff778b2793e0787acb6c551f8c2a
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Role "Lender" Role "Lender"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Role "Lender"
            (Party Role "Borrower")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
                  (Deposit Role "Borrower" Role "Borrower"
                     (Token "" "")
                     (Constant 2000000))
                  (Pay Role "Borrower"
                     (Party Role "Lender")
                     (Token "" 

### Transaction 3. Counterparty repays the loan's interest

The counterparty repays the interest.

In [105]:
TX_3=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party Borrower \
  --to-party Borrower \
  --lovelace "$((INTEREST*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$COUNTERPARTY_ADDR" \
  --address "$COUNTERPARTY_ADDR" \
  --manual-sign deposit-3.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_3 = $TX_3"

TX_3 = eb676d43d0a7bd13b91beaedb8ac2d204acf75d6970ccc3583157a4db61dfd7d


Sign the transaction.

In [106]:
cardano-cli transaction sign \
  --tx-body-file deposit-3.txbody \
  --out-file     deposit-3.tx \
  --signing-key-file "$COUNTERPARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [107]:
marlowe submit deposit-3.tx

"{\"blockHeaderHash\":\"179a53381f34f86a56f818f405a92c8339ca7299a61e0857166be5e9e1448450\",\"blockNo\":322735,\"slotNo\":6980662}"


View the contract's UTxO.

In [108]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_3//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
eb676d43d0a7bd13b91beaedb8ac2d204acf75d6970ccc3583157a4db61dfd7d     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "42e18c33625c28d765b400c24135d8a3918bf9c0a5b0d85fe4f29d0c7655bb3f"


View the party's UTxOs.

In [109]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253     4        1034400 lovelace + 1 567a873dff9bad13eb231829ebe26ad545862078c5251b8ef84dac9f.4c656e646572 + TxOutDatumNone
e4dd26d440107de4af57274f640bde39bebd0364ceb43fbdaf86346187bf3736     0        144586942 lovelace + TxOutDatumNone
e4dd26d440107de4af57274f640bde39bebd0364ceb43fbdaf86346187bf3736     2        1034400 lovelace + 1 42e84366ff974676655faf4f99f917145a12d29335ecb8162e6f843c.4c656e646572 + TxOutDatumNone


View the counterparty's UTxOs.

In [110]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
b2a9bca0c5481983ea8c63aa78a29b91ef29f608e50b48553b3f5c8e88a33d1c     1        2000000 lovelace + 1 567a873dff9bad13eb231829ebe26ad545862078c5251b8ef84dac9f.426f72726f776572 + TxOutDatumNone
eb676d43d0a7bd13b91beaedb8ac2d204acf75d6970ccc3583157a4db61dfd7d     0        247227684 lovelace + TxOutDatumNone
eb676d43d0a7bd13b91beaedb8ac2d204acf75d6970ccc3583157a4db61dfd7d     2        1043020 lovelace + 1 42e84366ff974676655faf4f99f917145a12d29335ecb8162e6f843c.426f72726f776572 + TxOutDatumNone


View the contract's history.

In [111]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253 (creation)
ContractId:      7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253#1
SlotNo:          6980558
BlockNo:         322731
BlockId:         51785845562e3493647637e398775d62f70bff778b2793e0787acb6c551f8c2a
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Role "Lender" Role "Lender"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Role "Lender"
            (Party Role "Borrower")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
                  (Deposit Role "Borrower" Role "Borrower"
                     (Token "" "")
                     (Constant 2000000))
                  (Pay Role "Borrower"
                     (Party Role "Lender")
                     (Token "" 

### Transaction 4. Counterparty repays the loan's principal

The counterparty repays the principal.

In [112]:
TX_4=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party Borrower \
  --to-party Borrower \
  --lovelace "$((PRINCIPAL*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$COUNTERPARTY_ADDR" \
  --address "$COUNTERPARTY_ADDR" \
  --manual-sign deposit-4.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_4 = $TX_4"

TX_4 = fa43d0b49047ace9b32f44944989c68b6b0db8615bfd1559277558ffde6b23da


Sign the transaction.

In [113]:
cardano-cli transaction sign \
  --tx-body-file deposit-4.txbody \
  --out-file     deposit-4.tx \
  --signing-key-file "$COUNTERPARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [114]:
marlowe submit deposit-4.tx

"{\"blockHeaderHash\":\"584f926987508abbf086688d3e1bf9853cdda478565cf46f39db5f8370a982a6\",\"blockNo\":322736,\"slotNo\":6980746}"


Check that there is no UTxO for the contract.

In [115]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_4//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


View the party's UTxOs.

In [116]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253     4        1034400 lovelace + 1 567a873dff9bad13eb231829ebe26ad545862078c5251b8ef84dac9f.4c656e646572 + TxOutDatumNone
e4dd26d440107de4af57274f640bde39bebd0364ceb43fbdaf86346187bf3736     0        144586942 lovelace + TxOutDatumNone
e4dd26d440107de4af57274f640bde39bebd0364ceb43fbdaf86346187bf3736     2        1034400 lovelace + 1 42e84366ff974676655faf4f99f917145a12d29335ecb8162e6f843c.4c656e646572 + TxOutDatumNone
fa43d0b49047ace9b32f44944989c68b6b0db8615bfd1559277558ffde6b23da     3        3000000 lovelace + TxOutDatumNone


View the counterparty's UTxOs.

In [117]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
b2a9bca0c5481983ea8c63aa78a29b91ef29f608e50b48553b3f5c8e88a33d1c     1        2000000 lovelace + 1 567a873dff9bad13eb231829ebe26ad545862078c5251b8ef84dac9f.426f72726f776572 + TxOutDatumNone
fa43d0b49047ace9b32f44944989c68b6b0db8615bfd1559277558ffde6b23da     0        146555284 lovelace + TxOutDatumNone
fa43d0b49047ace9b32f44944989c68b6b0db8615bfd1559277558ffde6b23da     1        1043020 lovelace + 1 42e84366ff974676655faf4f99f917145a12d29335ecb8162e6f843c.426f72726f776572 + TxOutDatumNone


View the contract's history.

In [118]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253 (creation)
ContractId:      7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253#1
SlotNo:          6980558
BlockNo:         322731
BlockId:         51785845562e3493647637e398775d62f70bff778b2793e0787acb6c551f8c2a
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Role "Lender" Role "Lender"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Role "Lender"
            (Party Role "Borrower")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
                  (Deposit Role "Borrower" Role "Borrower"
                     (Token "" "")
                     (Constant 2000000))
                  (Pay Role "Borrower"
                     (Party Role "Lender")
                     (Token "" 

## Cleanup

Remove the contract from history tracking.

In [119]:
marlowe rm "$CONTRACT_ID"

7f0428a9456f56fcac18d7f5b6ef90f3e6b9b22fbbc7459c67a218a49dedc253#1


In [120]:
marlowe ls

Consolidate the UTxOs at the addresses.

In [121]:
marlowe-cli util clean \
  --change-address "$PARTY_ADDR" \
  --required-signer "$PARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "487370a377bbeb4ccf08bfac21b406d87b892e6cdddae46a3615a07c936d9c09"


In [122]:
marlowe-cli util clean \
  --change-address "$COUNTERPARTY_ADDR" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "00aa78921bdbf83140390af828bb8c325628cf4bb0d84d21f4512c08e132f8d0"


Send the funds back to the faucet.

In [123]:
marlowe-cli transaction simple \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$PARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$COUNTERPARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --required-signer "$PARTY_SKEY" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --change-address "$FAUCET_ADDR" \
  --out-file /dev/null \
  --submit 600

TxId "fba80e3f2ab226d30f697af3f8ec3850ac885f95eab8be16ceb9dc0d1fd792d2"


See that the funds have been returned to the faucet.

In [124]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
00aa78921bdbf83140390af828bb8c325628cf4bb0d84d21f4512c08e132f8d0     1        2000000 lovelace + 1 42e84366ff974676655faf4f99f917145a12d29335ecb8162e6f843c.426f72726f776572 + TxOutDatumNone
00aa78921bdbf83140390af828bb8c325628cf4bb0d84d21f4512c08e132f8d0     2        2000000 lovelace + 1 567a873dff9bad13eb231829ebe26ad545862078c5251b8ef84dac9f.426f72726f776572 + TxOutDatumNone
487370a377bbeb4ccf08bfac21b406d87b892e6cdddae46a3615a07c936d9c09     1        2000000 lovelace + 1 42e84366ff974676655faf4f99f917145a12d29335ecb8162e6f843c.4c656e646572 + TxOutDatumNone
487370a377bbeb4ccf08bfac21b406d87b892e6cdddae46a3615a07c936d9c09     2        2000000 lovelace + 1 567a873dff9bad13eb231829ebe26ad545862078c5251b8ef84dac9f.4c656e646572 + TxOutDatumNone


Note that we don't have a way at this time to burn the role tokens! So they'll be sitting in these addresses.